# CNNs pentru clasificarea textelor

Folosirea CNN-urilor pentru a clasifica textul a fost prima dată prezentată în: [Convolutional Neural Networks for Sentence Classification](https://aclanthology.org/D14-1181.pdf).

### Arhitectura CNN

<img src="https://richliao.github.io/images/YoonKim_ConvtextClassifier.png">

Dându-se ca input un text de $n$ cuvinte $w_{1}$, $w_{2}$, ..., $w_{n}$, transformăm fiecare cuvânt într-un vector de dimensiune $d$, rezultând vectorii $w_{1}$, $w_{2}$, ..., $w_{n}$ aparținând $R^d$. Matricea rezultată de dimensiune $d$×$n$ este apoi folosită ca input pentru un layer convoluțional care trece un *sliding window* peste text.

Pentru fiecare window de lungime $l$:

$u_{i}$ = [$w_{i}$, ..., $w_{i+l-1}$] $∈ R^{d×l}$, 0≤$i$≤$n-l$ 

Pentru fiecare filtru $f_{j} ∈ R^{d×l}$ calculăm <$u_{i}$, $f_{j}$> și obținem matricea $ F ∈ R^{m×n}$ (dacă am făcut padding înainte de aplicarea filtrului, astfel încât să păstrăm dimensiunea $n$ a cuvintelor), unde $m$ este numărul de filtre. Aplicăm max-pooling pe matricea $F$ rezultată, apoi aplicăm funcția de activare. În final, avem un layer *fully connected* care produce distribuția pe clase, din care rezultă clasa cu cea mai mare probabilitate.

### Convoluții și filtre

<img src="https://debajyotidatta.github.io/assets/images/conv.001.png" width="300">

**Inputul** este format dintr-o matrice de dimensiune $n$×$d$, unde $n$ este numărul de cuvinte sau caractere, iar $d$ este lungimea reprezentării vectoriale sau lungimea vocabularului.  

De exemplu, pentru reprezentarea vectorială a unui text la nivel de caracter, pentru $d$ = 70, numărul de caractere unice în vocabular, pe fiecare linie a matricei avem reprezentarea one-hot a unui caracter.

<img src="https://debajyotidatta.github.io/assets/images/conv.002.png" width="500">

**Filtrele** (kernels) pot avea orice lungime. Lungimea este dată de numărul de linii din filtru. Lățimea filtrului trebuie să fie aceeași cu numărul de coloane din reprezentarea vectorială ($d$).

<img src="https://debajyotidatta.github.io/assets/images/conv.003.png" width="300">

Operația de convoluție presupune multiplicarea elementelor din input și filtru, rezultând o valoare care reprezintă suma rezultatelor multiplicării. În consecință, operația de convoluție multiplică *weight*-urile din filtru cu reprezentarea vectorială a cuvintelor.

<img src="https://debajyotidatta.github.io/assets/images/conv.004.png" width="300">

<img src="https://debajyotidatta.github.io/assets/images/conv.005.png" width="300">

<img src="https://debajyotidatta.github.io/assets/images/conv.006.png" width="300">

Filtrul este aplicat secvențial peste input, dar, la fel ca în cazul imaginilor, putem folosi diferite valori pentru *stride* pentru a controla cât de mult de mișcă filtrul vertical. Utilizând *stride* cu o valoare $k$ putem aplica un filtrul din $k$ în $k$ linii. De exemplu, pentru un filtru cu stride = 2, filtrul va fi aplicat pe secvența de text din 2 în 2 linii și vom avea un output de dimensiune mai mică.

<img src="https://debajyotidatta.github.io/assets/images/conv2.006.png" width="450">

Filtre multiple vor produce output-uri multiple. 

<img src="https://debajyotidatta.github.io/assets/images/conv2.007.png" width="450">

La următorul pas se realizează *max pooling* peste fiecare feature map rezultat din aplicarea filtrelor, iar apoi rezultatele sunt concatenate. 



### Imagini vs Text

Pentru a înțelege de ce o abordare folosind CNN-uri este potrivită pentru text, trebuie să ne gândim la textele noastre ca fiind niște imagini.

Pentru exemplul următor vom cosidera că reprezentarea unei propoziții a fost făcută la nivel de cuvânt.

De exemplu, pentru o propoziție cu lungimea maximă de 70 de cuvinte și lungimea embeddingului egală cu 300, putem crea o matrice cu valori numerice de forma 70x300 pentru a reprezenta această propoziție. Spre deosebire de imagini, în care elementele matricei sunt reprezentate de valori ale pixelilor, fiecare linie din reprezentarea vectorială a propoziției este, de fapt, reprezentarea unui cuvânt.

În cazul imaginilor, filtrul de convoluție se deplasează și vertical și orizontal, dar în cazul textului, filtrul se deplasează doar vertical, convoluțiile sunt doar 1D. Un kernel de dimensiune (2, 300), care are dimensiunea filtrului egală cu 2 se uită doar la 2 cuvinte în același timp. Ne putem gândi, deci, la dimensionea filtrelor ca la o dimensiune a n-gramelor (bigrame, trigrame, etc.).

În acest laborator vom folosi datasetul IMDb movie reviews: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [ ]:
! pip install unidecode

In [ ]:
import torch
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split
from unidecode import unidecode
from collections import Counter
import nltk
from nltk import word_tokenize
nltk.download('punkt')


In [ ]:
from urllib.request import urlretrieve
urlretrieve('https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv', 'IMDB_Dataset.csv')

In [ ]:
data = pd.read_csv('IMDB_Dataset.csv')
data = data[:10000]
data

Impartim datasetul in train si test.

In [ ]:
train_df, test_df = train_test_split(data, test_size=0.20, random_state = 42)

print('Dimensiunea datelor de train', len(train_df))
print('Dimensiunea datelor de test', len(test_df))

Așa cum am văzut în laboratoarele trecute, nu putem antrena un model direct pe datele sub formă de text, trebuie să transformam datele în reprezentări numerice vectoriale. 

Pentru asta, trebuie să parcurgem 2 pași:

- **Tokenizare**: împărțirea textelor în subtexte mai mici. Astfel vom determina vocabularul setului nostru de date (setul de tokeni unici)

- **Vectorizare**: reprezentarea în format numeric vectorial

Textul poate fi reprezentat fie ca o secvență de caractere, fie ca o secvență de cuvinte. Utilizarea reprezentării la nivel de cuvânt are o performanță mai bună și este mai folosită, pe când reprezentarea la nivel de caracter este utilă dacă textele au multe greșeli de scriere. 

### Reprezentarea vectorială la nivel de caracter


```
Texts: 'the mouse ran up the clock' and 'the mouse ran down'
```

Pe langa caracterele prezente in textele noastre, adaugam si 2 tokeni speciali: UNK (unknown) si PAD.


```
Index assigned for every token: {0: 'UNK', 1: 'PAD', 2: 't', 3: 'm', 4: 'c', 5: 'h', 6: 'l', 7: 'w', 8: ' ', 9: 'a', 10: 'k', 11: 'e', 12: 'r', 13: 'u', 14: 'n', 15: 's', 16: 'd', 17: 'p', 18: 'o'}
```

Reprezentarea vectoriala a celor doua texte folosind indexul corespunzator pentru fiecare caracter:

```
'the mouse ran up the clock' = [2, 5, 11, 8, 3, 18, 13, 15, 11, 8, 12, 9, 14, 8, 13, 17, 8, 2, 5, 11, 8, 4, 6, 18, 4, 10]
'the mouse ran down' = [2, 5, 11, 8, 3, 18, 13, 15, 11, 8, 12, 9, 14, 8, 16, 18, 7, 14]
```

Adaugam valori de padding la cel de-al doilea vector pentru a avea o lungime egala cu primul vector si obtinem:

```
[2, 5, 11, 8, 3, 18, 13, 15, 11, 8, 12, 9, 14, 8, 16, 18, 7, 14, 1, 1, 1, 1, 1, 1, 1, 1]
```

Transformăm lista de review-uri într-o listă de caractere pentru fiecare review.

In [ ]:
def transform_to_char(data):

    reviews = []
    
    for review in data:
        review_cleaned = [char.lower() for char in review]
        reviews.append(review_cleaned)

    return reviews

train_reviews = transform_to_char(train_df.review)
train_labels = train_df.sentiment.tolist()

print('Reviews', len(train_reviews))
print('Labels', len(train_labels))

Calculam marimea vocabularului de caractere.

In [ ]:
def get_vocab(data):

    units = set([unit for review in data for unit in review])
    
    return units

vocab = get_vocab(train_reviews)

print('total chars:', len(vocab))
print(vocab)

Putem vedea ca avem foarte multe caractere cu accente, diferite de caracterele limbii engleze. Pentru a micsora lungimea vocabularului, putem transforma caracterele utf8 in cea mai apropiata forma ASCII a lor.

In [ ]:
reviews_to_ascii = [unidecode(review) for review in train_df.review]
train_reviews = transform_to_char(reviews_to_ascii)
vocab = get_vocab(train_reviews)

print('total chars:', len(vocab))
print(vocab)

Atribuim fiecarui caracter din vocabularul nostru un index. Vom atribui 0 caracterelor necunoscute, iar 1 va fi valoarea atribuita paddingului.

In [ ]:
char_indices = dict((c, i + 2) for i, c in enumerate(vocab))
indices_char = dict((i + 2, c) for i, c in enumerate(vocab))

indices_char[0] = 'UNK'
char_indices['UNK'] = 0

indices_char[1] = 'PAD'
char_indices['PAD'] = 1

print('Dimensiunea vocabularului', len(indices_char))
print(indices_char)

Acum putem transforma propozitiile din datasetul nostru intr-o reprezentare vectoriala, in care vom avea pentru fiecare caracter indicele corespunzator din vocabularul nostru.

In [ ]:
import numpy as np

def vectorize_sentences(data, char_indices, one_hot = False):
    vectorized = []
    for sentences in data:

        # transformam fiecare review in reprezentarea lui sub forma de indici ale caracterelor continute
        sentences_of_indices = [char_indices[w] if w in char_indices.keys() else char_indices['UNK'] for w in sentences]

        # pentru fiecare indice putem face reprezentarea one-hot corespunzatoare
        # sau putem sa nu facem asta si sa adaugam un embedding layer in model care face această transformare
        if one_hot:
            sentences_of_indices = np.eye(len(char_indices))[sentences_of_indices]

        vectorized.append(sentences_of_indices)

    return vectorized

train_reviews_vectorized = vectorize_sentences(train_reviews, char_indices)
# train_reviews_vectorized[0]

In [ ]:
vectors_dim = [len(repr) for repr in train_reviews_vectorized]
vectors_dim[:5]

Putem vedea ca deoarece review-urile au numar diferit de caractere, in consecinta, si dimensiunile reprezentarilor vectoriale sunt diferite. 
Vom aduce reprezentarile noastre la aceeasi dimensiune maxima.

Definim o functie *pad* care:

- primeste un set de review-uri si o lungime maxima
- scurteaza toate reprezentarile mai mari decat lungimea maxima
- adauga valoarea de padding (1 in cazul nostru) la reprezentarile mai scurte decat lungimea maxima

In [ ]:
def pad(samples, max_length):
    
    return torch.tensor([
        sample[:max_length] + [1] * max(0, max_length - len(sample))
        for sample in samples
    ])

In [ ]:
train_reviews_vectorized = pad(train_reviews_vectorized, max_length = 1000)
train_reviews_vectorized.shape

In [ ]:
train_reviews_vectorized.shape

Tranformăm și review-urile din setul de test într-o reprezentare vectorială

In [ ]:
test_reviews_to_ascii = [unidecode(review) for review in test_df.review]
test_reviews = transform_to_char(test_reviews_to_ascii)
test_labels = test_df.sentiment.tolist()

print('Reviews', len(test_reviews))
print('Labels', len(test_labels))

In [ ]:
test_reviews_vectorized = vectorize_sentences(test_reviews, char_indices)
test_reviews_vectorized = pad(test_reviews_vectorized, max_length = 1000)
# test_reviews_vectorized[0]

Vom incarca seturile noastre de date intr-un obiect din clasa [Dataset](https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset).

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, samples, labels):
        self.samples = samples
        self.labels = labels
            
    def __getitem__(self, k):
        """Returneaza al k-lea exemplu din dataset"""
        return self.samples[k], self.labels[k]
    
    def __len__(self):
        """Returneaza dimensiunea datasetului"""
        return len(self.samples)

Definim arhitectura modelului

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        # Definim un embedding layer cu un vocabular de dimensiune 72
        # și ca output un embedding de dimensiune 20
        # padding_idx este indexul din vocabular al paddingului (1, în cazul nostru)
        
        self.embedding = torch.nn.Embedding(72, 20, padding_idx=1)

        # Definim o secvență de layere
        
        # Un layer Convolutional 1D cu 20 input channels, 32 output channels, dimensiune kernel = 3 și padding = 1
        # ReLU activation
        # 1D Maxpooling layer de dimensiune 2
        conv1 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=20, out_channels=32, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2),
        )
        
        # Un layer Convolutional 1D cu 32 input channels, 32 output channels, dimensiune kernel = 5 și padding = 2
        # ReLU activation
        # 1D Maxpooling layer de dimensiune 2
        conv2 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=32, out_channels=32, kernel_size=5, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2),
        )
        
        # Global Average pooling layer care, în cazul nostru, este un 1D Avgerage Pooling layer
        # cu dimensiunea de 250 și stride 250
        global_average = torch.nn.AvgPool1d(kernel_size=250, stride=250)

        self.convolutions = torch.nn.Sequential(
            conv1, conv2, global_average
        )
        
        # Flattening layer
        flatten = torch.nn.Flatten()
        
        # Linear layer cu 32 input features și 2 outputs fără funcție de activare
        linear = torch.nn.Linear(in_features=32, out_features=2)

        self.classifier = torch.nn.Sequential(flatten, linear)
        
    def forward(self, input):
        # trecem inputul prin layerul de embedding
        embeddings = self.embedding(input)
        
        # permutăm inputul astfel încât prima dimensiune este numărul de channels
        embeddings = embeddings.permute(0, 2, 1)
        
        # trecem inputul prin secvența de layere
        output = self.convolutions(embeddings)
        output = self.classifier(output)
        return output


In [ ]:
DEVICE = torch.device("cuda")
# instanțiem modelul
model = Model().to(DEVICE)

# Adam optimizer cu lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Cross Entropy loss
loss_fn = torch.nn.CrossEntropyLoss()

# training dataset and dataloader
# test dataset and dataloader
train_ds = Dataset(train_reviews_vectorized, train_labels)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
test_ds = Dataset(test_reviews_vectorized, test_labels)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=False)

Training loop

In [ ]:
best_val_acc = 0
for epoch_n in range(10):
    print(f"Epoch #{epoch_n + 1}")
    model.train()
    for batch in train_dl:
        model.zero_grad()

        inputs, targets = batch
        inputs = inputs.long().to(DEVICE)
        targets = targets.to(DEVICE)

        output = model(inputs)
        loss = loss_fn(output, targets)

        loss.backward()
        optimizer.step()

    # validare
    model.eval()
    all_predictions = torch.tensor([])
    all_targets = torch.tensor([])
    for batch in test_dl:
        inputs, targets = batch
        inputs = inputs.long().to(DEVICE)
        targets = targets.to(DEVICE)

        with torch.no_grad():
            output = model(inputs)

        predictions = output.argmax(1)
        all_targets = torch.cat([all_targets, targets.detach().cpu()])
        all_predictions = torch.cat([all_predictions, predictions.detach().cpu()])

    val_acc = (all_predictions == all_targets).float().mean().numpy()
    print(val_acc)

    if val_acc > best_val_acc:
        torch.save(model.state_dict(), "./model")
        best_val_acc = val_acc

print("Best validation accuracy", best_val_acc)

### Reprezentarea vectoriala la nivel de cuvant



```
Texts: 'The mouse ran up the clock' and 'The mouse ran down'
```

Pe langa tokenii prezenti in textele noastre, adaugam si 2 tokeni speciali: UNK (unknown word) si PAD.


```
Index assigned for every token: {'UNK': 0, 'PAD': 1, 'the': 2, 'mouse': 3, 'ran': 4, 'up': 5, 'clock': 6, 'down': 7}
```

Reprezentarea vectoriala a celor doua texte folosind indexul corespunzator pentru fiecare cuvant:

```
'The mouse ran up the clock' = [2, 3, 4, 5, 2, 6]
'The mouse ran down' = [2, 3, 4, 7]
```

Adaugam valori de padding la cel de-al doilea vector pentru a avea o lungime egala cu primul vector si obtinem:

```
[2, 3, 4, 7, 1, 1]
```
Reprezentarea one-hot a fiecarui text:

```
'The mouse ran up the clock' = [[0. 0. 1. 0. 0. 0. 0.]
                                [0. 0. 0. 1. 0. 0. 0.]
                                [0. 0. 0. 0. 1. 0. 0.]
                                [0. 0. 0. 0. 0. 1. 0.]
                                [0. 0. 1. 0. 0. 0. 0.]
                                [0. 0. 0. 0. 0. 0. 1.]]

'The mouse ran down' = [[0. 0. 1. 0. 0. 0. 0. 0.]
                        [0. 0. 0. 1. 0. 0. 0. 0.]
                        [0. 0. 0. 0. 1. 0. 0. 0.]
                        [0. 0. 0. 0. 0. 0. 0. 1.]
                        [0. 1. 0. 0. 0. 0. 0. 0.]]
```

Impartim textele din dataset in tokeni.

In [ ]:
def transform_to_tokens(data):

    reviews = []
    for review in data:
        review_tokenized = word_tokenize(review.lower())
        reviews.append(review_tokenized)

    return reviews

In [ ]:
train_reviews = transform_to_tokens(train_df.review)
for r in train_reviews[:2]:
    print(r[:20])

Construim vocabularul de tokeni

In [ ]:
vocab = get_vocab(train_reviews)

print('total words:', len(vocab))
print(list(vocab)[:100])

Avem un vocabular foarte mare. Vom scoate din vocabular cuvintele cu o frecventa foarte mica. 

O alta abordare pentru micscorarea vocabularului este scoaterea cuvintelor foarte frecvente (stopwords). De asemenea putem face si alti pasi de preprocesare: putem face stemming, lematizare, putem scoate punctuatia, etc.)

In [ ]:
import operator

def word_freq(data, min_aparitions):
    
    all_words = [words.lower() for sentences in data for words in sentences]
    sorted_vocab = sorted(dict(Counter(all_words)).items(), key=operator.itemgetter(1))
    final_vocab = [k for k,v in sorted_vocab if v > min_aparitions]

    return final_vocab

In [ ]:
vocab = word_freq(train_reviews, min_aparitions = 10)

print(vocab[:100])
print(len(vocab))

Fiecarui token ii atribuim un indice.

In [ ]:
word_indices = dict((c, i + 2) for i, c in enumerate(vocab))
indices_word = dict((i + 2, c) for i, c in enumerate(vocab))

indices_word[0] = 'UNK'
word_indices['UNK'] = 0

indices_word[1] = 'PAD'
word_indices['PAD'] = 1

In [ ]:
# print(indices_word)

Acum putem transforma propozitiile din datasetul nostru intr-o reprezentare vectoriala, in care vom avea pentru fiecare cuvant indicele corespunzator din vocabular.

In [ ]:
train_reviews_vectorized = vectorize_sentences(train_reviews, word_indices)

In [ ]:
vectors_dim = [len(repr) for repr in train_reviews_vectorized]
vectors_dim[:5]

Din nou, pentru ca textele au un numar diferit de cuvinte, trebuie sa aducem vectorii la o reprezentare de aceeași dimensiune.

In [ ]:
train_reviews_vectorized = pad(train_reviews_vectorized, max_length = 512)
train_reviews_vectorized

In [ ]:
train_reviews_vectorized.shape

# TASK
## Deadline: 31 martie ora 23:59.

Formular pentru trimiterea temei: https://forms.gle/Bznaciv2MTy4kVL47

Folosind intreg datasetul de mai sus (IMDb reviews) implementati urmatoarele cerinte:
1. Impartiti setul de date in 80% train, 10% validare si 10% test
2. Tokenizati textele si determinati vocabularul (in acest task vom lucra cu reprezentari la nivel de cuvant, NU la nivel de caracter); intrucat vocabularul poate fi foarte mare, incercati sa aplicati una dintre tehnicile mentionate in laborator (10K-20K de cuvinte ar fi o dimensiunea rezonabila a vocabularului)
3. Transformati textele in vectori de aceeasi dimensiune folosind indexul vocabularului (alegeti o dimensiune maxima de circa 500-1000 de tokens)
4. Implementati urmatoarea arhitectura:
    * un Embedding layer pentru vocabularul determinat, ce contine vectori de dimensiune 100
    * un layer dropout cu probabilitate 0.4
    * un layer convolutional 1D cu 100 canale de input si 128 de canale de output, dimensiunea kernelului de 3 si padding 1; asupra rezultatului aplicati un layer de [BatchNormalization](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html) cu 128 features; aplicati apoi functia de activare ReLU, iar in cele din urma un strat de max-pooling 1D cu kernel size 2.
    * un layer convolutional 1D cu 128 canale de input si 128 de canale de output, dimensiunea kernelului de 5 si padding 2; asupra rezultatului aplicati un layer de BatchNormalization cu 128 features; aplicati apoi functia de activare ReLU, iar in cele din urma un strat de max-pooling 1D cu kernel size 2.
    * un layer convolutional 1D cu 128 canale de input si 128 de canale de output, dimensiunea kernelului de 5 si padding 2; asupra rezultatului aplicati un layer de BatchNormalization cu 128 features; aplicati apoi functia de activare ReLU, iar in cele din urma un strat de max-pooling 1D cu kernel size 2.
    * asupra rezultatului ultimului layer, aplicati average-pooling 1D obtinand pentru fiecare canal media tuturor valorilor din vectorul sau corespunzator
    * un layer feed-forward (linear) cu dimensiunea inputului 128, si 2 noduri pentru output (pentru clasificare in 0/1)
5. Antrenati arhitectura folosind cross-entropy ca functie de loss si un optimizer la alegere. La finalul fiecarei epoci evaluati modelul pe datele de validare si salvati weighturile celui mai bun model astfel determinat
6. Evaluati cel mai bun model obtinut pe datele de test.


1. Impartiti setul de date in 80% train, 10% validare si 10% test

In [1]:
# citim datele, si le convertim la un format mai usor
# de procesat
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('IMDB_Dataset.csv')
data = list(zip(data.review, data.sentiment))

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

print(f"Train data: {len(train_data)}")
print(f"Test data:  {len(test_data)}")
print(f"Val data:   {len(val_data)}")

Train data: 40000
Test data:  5000
Val data:   5000


2. Tokenizati textele si determinati vocabularul (in acest task vom lucra cu reprezentari la nivel de cuvant, NU la nivel de caracter); intrucat vocabularul poate fi foarte mare, incercati sa aplicati una dintre tehnicile mentionate in laborator (10K-20K de cuvinte ar fi o dimensiunea rezonabila a vocabularului)

In [2]:
from unidecode import unidecode
import re
import num2words
from typing import List, Tuple

def remove_special_characters(s: str):
    """
        Scoate toate caracterele ne-ascii, si tot ce nu e cuvant este
        inlocuit cu un spatiu.
    """
    # scoatem caracterele speciale
    s = unidecode(s)

    # inlocuim numerele
    s = re.sub(r'\d+', lambda s: ' ' + num2words.num2words(s.group(), lang='en') + ' ', s)
    
    # convertim la lowercase
    s = s.lower()

    # scoatem tot ce este punctuatie
    s = re.sub(r'\W+', ' ', s)
    
    return s

def remove_special_characters_arr(s: List[Tuple[str, int]]):
    return list(map(lambda s: (remove_special_characters(s[0]).split(), s[1]), s))

In [3]:
train_fixed_str = remove_special_characters_arr(train_data)
val_fixed_str = remove_special_characters_arr(val_data)
test_fixed_str = remove_special_characters_arr(test_data)

print(f"train_fixed_str[0] = {train_fixed_str[0]}")

train_fixed_str[0] = (['lame', 'plot', 'and', 'two', 'dimensional', 'script', 'made', 'characters', 'look', 'like', 'cardboard', 'cut', 'outs', 'needless', 'to', 'say', 'this', 'made', 'it', 'difficult', 'to', 'feel', 'empathy', 'for', 'any', 'of', 'the', 'characters', 'especially', 'the', 'fiance', 'he', 'looked', 'and', 'acted', 'more', 'like', 'a', 'cartoon', 'in', 'summary', 'i', 'guess', 'you', 'could', 'say', 'it', 'was', 'on', 'par', 'with', 'your', 'typical', 'made', 'for', 'tv', 'drama', 'it', 'uses', 'just', 'about', 'every', 'cliche', 'in', 'the', 'book', 'the', 'tortured', 'classical', 'musician', 'who', 'wants', 'to', 'break', 'out', 'and', 'play', 'salsa', 'the', 'free', 'spirited', 'fiancee', 'engaged', 'to', 'a', 'bean', 'counter', 'personality', 'she', 'doesn', 't', 'love', 'i', 'won', 't', 'list', 'them', 'or', 'else', 'it', 'would', 'be', 'a', 'spoiler', 'because', 'i', 'd', 'be', 'giving', 'away', 'the', 'whole', 'plot', 'the', 'dancing', 'was', 'ok', 'but', 'nothin

In [4]:
from collections import Counter

# frecventa fiecarui cuvant in train
words_frq = Counter([s for point in train_fixed_str for s in point[0]])

# vocabularul pe care il folosim
VOCAB_WORDS_SIZE = 10000
vocab = sorted(list(words_frq), key=lambda e: -words_frq[e])
# scadem 2 ca sa avem si 'unk' si 'pad'
vocab = vocab[:VOCAB_WORDS_SIZE - 2]

word_indices = dict((c, i + 2) for i, c in enumerate(vocab))
indices_word = dict((i + 2, c) for i, c in enumerate(vocab))

indices_word[0] = 'UNK'
word_indices['UNK'] = 0

indices_word[1] = 'PAD'
word_indices['PAD'] = 1

print(f"First items of vocab: {vocab[:30]}")

First items of vocab: ['the', 'and', 'a', 'of', 'to', 'is', 'br', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'movie', 'for', 'with', 'but', 'film', 'one', 'you', 't', 'on', 'not', 'he', 'are', 'his', 'have', 'be']


3. Transformati textele in vectori de aceeasi dimensiune folosind indexul vocabularului (alegeti o dimensiune maxima de circa 500-1000 de tokens)

In [5]:
import torch as th

# spargem labelul de text
train_labels = list(map(lambda s: s[1], train_fixed_str))
val_labels = list(map(lambda s: s[1], val_fixed_str))
test_labels = list(map(lambda s: s[1], test_fixed_str))
train_str = list(map(lambda s: s[0], train_fixed_str))
val_str = list(map(lambda s: s[0], val_fixed_str))
test_str = list(map(lambda s: s[0], test_fixed_str))

word_to_index = lambda s: word_indices[s] if s in word_indices else 0
MAX_TOKENS = 500

def fix_dataset(s: List[List[str]]):
    fix_str = lambda s: s[:min(len(s), MAX_TOKENS)] + ['PAD'] * max(0, MAX_TOKENS - len(s))
    s = map(fix_str, s)
    s = [list(map(word_to_index, i)) for i in s]
    return th.tensor(s)

train_tokenized = fix_dataset(train_str)
val_tokenized = fix_dataset(val_str)
test_tokenized = fix_dataset(test_str)

print(f"train shape: {train_tokenized.shape}")
print(f"val shape: {val_tokenized.shape}")
print(f"test shape: {test_tokenized.shape}")

train shape: torch.Size([40000, 500])
val shape: torch.Size([5000, 500])
test shape: torch.Size([5000, 500])


4. Implementati urmatoarea arhitectura:
    * un Embedding layer pentru vocabularul determinat, ce contine vectori de dimensiune 100
    * un layer dropout cu probabilitate 0.4
    * un layer convolutional 1D cu 100 canale de input si 128 de canale de output, dimensiunea kernelului de 3 si padding 1; asupra rezultatului aplicati un layer de [BatchNormalization](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html) cu 128 features; aplicati apoi functia de activare ReLU, iar in cele din urma un strat de max-pooling 1D cu kernel size 2.
    * un layer convolutional 1D cu 128 canale de input si 128 de canale de output, dimensiunea kernelului de 5 si padding 2; asupra rezultatului aplicati un layer de BatchNormalization cu 128 features; aplicati apoi functia de activare ReLU, iar in cele din urma un strat de max-pooling 1D cu kernel size 2.
    * un layer convolutional 1D cu 128 canale de input si 128 de canale de output, dimensiunea kernelului de 5 si padding 2; asupra rezultatului aplicati un layer de BatchNormalization cu 128 features; aplicati apoi functia de activare ReLU, iar in cele din urma un strat de max-pooling 1D cu kernel size 2.
    * asupra rezultatului ultimului layer, aplicati average-pooling 1D obtinand pentru fiecare canal media tuturor valorilor din vectorul sau corespunzator
    * un layer feed-forward (linear) cu dimensiunea inputului 128, si 2 noduri pentru output (pentru clasificare in 0/1)

In [6]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.embedding = th.nn.Embedding(VOCAB_WORDS_SIZE, 100, padding_idx=1)

        self.net = nn.Sequential(
            # primul strat de nebunii
            nn.Dropout(0.4),
            nn.Conv1d(100, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2),

            # al doilea strat
            nn.Conv1d(128, 128, kernel_size=5, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2),

            # al treilea strat
            nn.Conv1d(128, 128, kernel_size=5, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.AvgPool1d(62),

            nn.Flatten(),
            nn.Linear(128, 2)
        )
        
    def forward(self, input):
        output = self.embedding(input)
        output = output.permute(0, 2, 1)
        output = self.net(output)
        return output

5. Antrenati arhitectura folosind cross-entropy ca functie de loss si un optimizer la alegere. La finalul fiecarei epoci evaluati modelul pe datele de validare si salvati weighturile celui mai bun model astfel determinat

In [10]:
# generam dataloaderele
train_dataset = th.utils.data.TensorDataset(train_tokenized, th.tensor(train_labels))
train_dataloader = th.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

val_dataset = th.utils.data.TensorDataset(val_tokenized, th.tensor(val_labels))
val_dataloader = th.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

test_dataset = th.utils.data.TensorDataset(test_tokenized, th.tensor(test_labels))
test_dataloader = th.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [21]:
DEVICE = th.device('cuda')

net = Model().to(DEVICE)

optimizer = th.optim.Adam(params=net.parameters())
loss_fn = nn.CrossEntropyLoss()

best_val_acc = 0.
best_net = None

In [22]:
import copy
from tqdm import tqdm

for epoch_n in range(10):
    print(f"Epoch #{epoch_n + 1}")
    net.train()
    for batch in tqdm(train_dataloader):
        net.zero_grad()

        inputs, targets = batch
        inputs = inputs.long().to(DEVICE)
        targets = targets.to(DEVICE)

        output = net(inputs)
        loss = loss_fn(output, targets)

        loss.backward()
        optimizer.step()

    # validare
    net.eval()
    all_predictions = th.tensor([])
    all_targets = th.tensor([])
    for batch in val_dataloader:
        inputs, targets = batch
        inputs = inputs.long().to(DEVICE)
        targets = targets.to(DEVICE)

        with th.no_grad():
            output = net(inputs)

        predictions = output.argmax(1)
        all_targets = th.cat([all_targets, targets.detach().cpu()])
        all_predictions = th.cat([all_predictions, predictions.detach().cpu()])

    val_acc = (all_predictions == all_targets).float().mean().numpy()
    print(val_acc)

    if val_acc > best_val_acc:
        best_net = copy.deepcopy(net.cpu())
        net.to(DEVICE)
        best_val_acc = val_acc

print("Best validation accuracy", best_val_acc)

Epoch #1


100%|██████████| 313/313 [00:11<00:00, 26.71it/s]


0.83
Epoch #2


100%|██████████| 313/313 [00:12<00:00, 24.88it/s]


0.7554
Epoch #3


...

0.8822
Epoch #9


100%|██████████| 313/313 [00:12<00:00, 25.21it/s]


0.9056
Epoch #10


100%|██████████| 313/313 [00:12<00:00, 24.46it/s]


0.7564
Best validation accuracy 0.9056


6. Evaluati cel mai bun model obtinut pe datele de test.

In [24]:
import sklearn.metrics as metrics

best_net.to(DEVICE)
best_net.eval()
all_predictions = th.tensor([])
all_targets = th.tensor([])
for batch in test_dataloader:
    inputs, targets = batch
    inputs = inputs.long().to(DEVICE)
    targets = targets.to(DEVICE)

    with th.no_grad():
        output = best_net(inputs)

    predictions = output.argmax(1)
    all_targets = th.cat([all_targets, targets.detach().cpu()])
    all_predictions = th.cat([all_predictions, predictions.detach().cpu()])


print(
    f"Accuracy: {metrics.accuracy_score(all_targets, all_predictions)}\n"
    f"Precision: {metrics.precision_score(all_targets, all_predictions, average='macro')}\n"
    f"Recall: {metrics.recall_score(all_targets, all_predictions, average='macro')}\n"
    f"F1: {metrics.f1_score(all_targets, all_predictions, average='macro')}\n"
)

Accuracy: 0.9148
Precision: 0.9150296484499778
Recall: 0.9150623297072501
F1: 0.9147998773118233



In [25]:
# salvam modelul
th.save(best_net.state_dict(), "./model")